In [35]:
!pip3 install spacy==2.3.0
!pip3 install spacy_langdetect
!pip3 install selenium
!apt-get install chromium-chromedriver
!pip3 install tqdm
!git clone https://github.com/Elbereth-Elentari/Library_search_for_topic.git

!python3 Library_search_for_topic/python/Library_search_for_topic.py

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (83.0.4103.61-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
fatal: destination path 'Library_search_for_topic' already exists and is not an empty directory.
